Одна из первых проблем, с которыми можно столкнуться при составлении тонального словаря - парсинг даты:)
Как правило все российские маркетплейсы (Озон, Ламода, Вайлдберриз и тд) шифруются и их отзывы-комментарии либо подгружаются через какое-то время, и их не видно в скачанном html-коде, либо ссылка на нужный класс с ревьюз обновляется спустя какое-то время, и если сегодня мой код работает, то завтра скачать отзыв не получится (это не точно, но выглядит так, будто нужный класс в html рандомно генерируется). Также некоторые сайты с отзывами в принципе не позволяют скачать их дату, потому что считают меня роботом и блокируют, либо предлагают пройти капчу (и fake user agent не справляется). А если использовать Selenium как советуют одногруппники, то Selenium использует браузер (например, Хром). Если кто-то другой с нуля будет запускать код с неустановленным Хромом, то код не запустится. Это тоже проблема((((

In [ ]:
# Установка Selenium и браузер Chrome для успешного парсинга
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 985 kB 14.7 MB/s 
     |████████████████████████████████| 358 kB 53.3 MB/s 
     |████████████████████████████████| 140 kB 52.0 MB/s 
     |████████████████████████████████| 58 kB 6.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get

In [ ]:
# Подготовка Selenium к работе
import sys, time
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


In [ ]:
# url = 'https://www.kinopoisk.ru/film/435/reviews/'
url = 'https://www.kinopoisk.ru'
driver.get(url)
print(driver.current_url)
# button = driver.find_element_by_xpath('//*[@id="__next"]/div[2]/div[2]/div[2]/div/div/div[1]/div/div/div/div/a[3]')
# print(button)

https://www.kinopoisk.ru/


In [ ]:
import requests
from bs4 import BeautifulSoup

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


с сайтв imdb про фильмы берем отзывы не некоторые фильмы. ссылка на негативные ревью (рейтинг 1 звезда) получается путем прибавления к основной ссылке ratingFilter

In [ ]:
url_arr = ['https://m.imdb.com/title/tt0126029/reviews',
'https://m.imdb.com/title/tt0298148/reviews',
'https://m.imdb.com/title/tt0413267/reviews',
'https://m.imdb.com/title/tt0892791/reviews',
'https://m.imdb.com/title/tt10731256/reviews',
'https://m.imdb.com/title/tt13327038/reviews',
'https://m.imdb.com/title/tt10648342/reviews',
'https://m.imdb.com/title/tt15791034/reviews',
'https://m.imdb.com/title/tt8093700/reviews',
'https://m.imdb.com/title/tt1877830/reviews',
'https://m.imdb.com/title/tt15807910/reviews',
'https://m.imdb.com/title/tt3070936/reviews']
positive_url = '?ratingFilter=10'
negative_url = '?ratingFilter=1'

парсинг данных: функция, которая вытаскиевает все тексты со страницы, убирает лишние \ (слеши)

In [ ]:
def get_reviews(url):
  response = requests.get(url)
  html = BeautifulSoup(response.text, 'html.parser') 
  html_all_reviews = html.find('div', {'id': 'reviews-container'})
  html_review_arr = html_all_reviews.find_all('div', {'class': 'text'})
  reviews = []
  for html_review in html_review_arr:
    review = html_review.text.replace('\\', '') 
    reviews.append(review)
  return reviews

positive_reviews = [] 
negative_reviews = []
for url in url_arr:
  positive_reviews.extend(get_reviews(url + positive_url))
  negative_reviews.extend(get_reviews(url + negative_url))

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 17.5 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=3b30bee110070fa94f07702c54eafccbe18459b86cd63f9fe90e39590905e05d
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [ ]:
import nltk
import pymorphy2
import nltk
nltk.download('book')
nltk.download('all-corpora')
nltk.download('punkt')

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/chat80.zip.
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package conll2000 to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/conll2000.zip.
[nltk_data]    | Downloading package conll2002 to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/conll2002.zip.
[nltk_data]    | Downloading package dependency_treebank to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/dependency_treebank.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    

True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

пишем функцию, которая приводит слово в нормальную (начальную) форму. (ex: were --> be).  

In [ ]:
lemmatizer = WordNetLemmatizer()
tag_dict = {"J": wordnet.ADJ,
            "N": wordnet.NOUN,
            "V": wordnet.VERB,
            "R": wordnet.ADV}

morph = pymorphy2.MorphAnalyzer()

def tokenize_and_normal_form(reviews):
  normal_forms = []
  for review in reviews:
    tokens = nltk.word_tokenize(review)
    tokens = nltk.pos_tag(tokens)
    for token in tokens:
      word = token[0].lower()
      tag = tag_dict.get(token[1][0], wordnet.NOUN)
      normal_form = lemmatizer.lemmatize(word, tag)
      normal_forms.append(normal_form)
  return normal_forms

positive_reviews_normal_form = tokenize_and_normal_form(positive_reviews)
negative_reviews_normal_form = tokenize_and_normal_form(negative_reviews)

убираем лишний частотный шум (стопслова), берем топ-100 популярных слов для + и - ревью 

In [ ]:
from collections import Counter
nltk.download("stopwords")
from nltk.corpus import stopwords
english_stopwords = stopwords.words("english")
def collect_freqlist(reviews, max_len=100):    
    freqlist = Counter()
    for word in reviews:
          if word.isalpha() and word not in english_stopwords:
              freqlist[word] += 1
    return dict(freqlist.most_common(max_len))

positive_most_common_words = collect_freqlist(positive_reviews_normal_form)
negative_most_common_words = collect_freqlist(negative_reviews_normal_form)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
set_positive_most_common_words = set(positive_most_common_words.keys())
set_negative_most_common_words = set(negative_most_common_words.keys())

only_positive_words = set_positive_most_common_words.difference(set_negative_most_common_words)
only_negative_words = set_negative_most_common_words.difference(set_positive_most_common_words)

In [ ]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=e11859c84588c6995d785fbc06143de10a220452079f950797b8059cd948932f
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [ ]:
from sklearn.metrics import accuracy_score

ф-ция которая определяет, какой перед нами ревью: положительный или отрицательный. используется простая схема: если в отзыве больше таких слов, что встречаются в положительных, значит этот ревью положительный. равное кол-во включено в негативные (в-ть такого случая слишком мала чтобы его в отдельный). 

In [ ]:
def simple_review_detect(review):
    result = []
    for word in review:
      if word in only_positive_words:
        result.append('positive')
      elif word in only_negative_words:
        result.append('negative')
    return 'positive' if result.count('positive') > result.count('negative') else 'negative'

ф-ция берет размеченный ревью (такие, что первый элемент = текст отзыва, второй - тег (+ или - отзыв). и сравнивает с золотым стандартом (верно размеченный отзыв) пример: [хороший фильм, советую; positive], считает точность 

In [ ]:
def accuracy(marked_all_reviews):
    predict = []  # сюда будем писать предсказания
    gold = []     # сюда будем писать действительные теги
    for marked_review in marked_all_reviews:
      review = marked_review[0]
      gold_tag = marked_review[1]
      predict_tag = simple_review_detect(review)
      gold.append(gold_tag)
      predict.append(predict_tag)
    print("RESULTS:")
    print("Accuracy: %.4f" % accuracy_score(predict, gold))

In [ ]:
gold_standart_url = ['https://m.imdb.com/title/tt0499549/reviews',
'https://m.imdb.com/title/tt12593682/reviews']

reviews = [] 
marked_all_reviews = []
for url in gold_standart_url:
  reviews = tokenize_and_normal_form(get_reviews(url + positive_url))
  for review in reviews:
    marked_all_reviews.append((review, 'positive'))

for url in gold_standart_url:
  reviews = tokenize_and_normal_form(get_reviews(url + negative_url))
  for review in reviews:
    marked_all_reviews.append((review, 'negative'))

import random
random.shuffle(marked_all_reviews)

In [ ]:
accuracy(marked_all_reviews)

RESULTS:
Accuracy: 0.4778


1)
увеличить количество данных для составления тонального словаря
2)
CBOW
можно учитывать контекст на примере алгоритма CBOW: смотреть на н-слов до и после. например, если до ключевого слова идет 'не' (not, no..), то такому токену будет присвоен тег положительного отзыва (а не отрицательного) и наоборот.